In [1]:
import drawsvg as draw
import numpy as np
import bezier
import matplotlib.pyplot as plt
import scipy as sp

In [2]:
smooth_x = 0.6
smooth_y = 0.84
width = 19.5
height = 75

edge_offset = 0.6
trace_width = 0.15
trace_spacing = trace_width
image_scale = 15

# True for default version, False for rotated version.
make_symmetric = True

# =====================================

assert smooth_x <= 1 and smooth_x >= 0
assert smooth_y <= 1 and smooth_y >= 0
assert width > 0
assert height > 0
assert edge_offset >= 0 and edge_offset < width/2


def calc_length(obj):
    segs = vars(p)["args"]["d"].split(" ")
    #print(segs)
    ops = [x[0] for x in segs]
    assert ops[0] == "M"
    assert "M" not in ops[1:]
    
    total_len = 0
    origin = [float(x) for x in segs[0][1:].split(",")]
    for seg in segs[1:]:
        nex = [float(x) for x in seg[1:].split(",")]
        if seg[0] == "L":
            total_len += np.sqrt( (nex[0]-origin[0])**2 + (nex[1]-origin[1])**2 )
            origin = nex
        elif seg[0] == "A":
            assert nex[0] == nex[1], "Not a circle"
            assert np.abs(np.abs(nex[5]-origin[0])-nex[0]) < 0.001, "Bad arclen"
            assert np.abs(np.abs(nex[6]-origin[1])-nex[1]) < 0.001, "Bad arclen"
            total_len += np.pi*nex[0]/2
            origin=[nex[5],nex[6]]
        else:
            assert False, seg[0]
    return total_len
        

d = draw.Drawing(800, 1000, origin=(-10,-10), displayInline=False)

nodes_left = np.asfortranarray([
[0, 0,               (width/2)-(width*smooth_x/2), width/2],
[0, height*smooth_y, height,             height]
])
nodes_right = np.asfortranarray([
[width, width,          (width/2)+(width*smooth_x/2), width/2],
[0,     height*smooth_y,height,                     height]
])

curve_left = bezier.Curve(nodes_left, degree=3)
curve_right = bezier.Curve(nodes_right, degree=3)
points_left = curve_left.evaluate_multi(np.linspace(0,1,100)).T
points_right = curve_right.evaluate_multi(np.linspace(0,1,100)).T


gradients_left = np.gradient(points_left.T[1],points_left.T[0].T)
gradients_right = np.gradient(points_right.T[1],points_right.T[0].T)

points_left_offset = []
for seg in list(zip(gradients_left,points_left[:-1])):
    dist = 20
    offset_x = np.sqrt(edge_offset*edge_offset/((1/seg[0])**2+1))
    offset_y = offset_x * (1/seg[0])
    points_left_offset.append([seg[1][0]+offset_x, seg[1][1]-offset_y])
    
    p = draw.Path(stroke_width = trace_width, stroke="purple", fill_opacity=0)
    p.M(*seg[1])
    p.L(*points_left_offset[-1])
    d.draw(p)
points_left_offset = np.array(points_left_offset)
    
points_right_offset = []
for seg in list(zip(gradients_right,points_right[:-1])):
    dist = 20
    offset_x = np.sqrt(edge_offset*edge_offset/((1/seg[0])**2+1))
    offset_y = offset_x * (1/seg[0])
    points_right_offset.append([seg[1][0]-offset_x, seg[1][1]+offset_y])
    
    p = draw.Path(stroke_width = trace_width, stroke="purple", fill_opacity=0)
    p.M(*seg[1])
    p.L(*points_right_offset[-1])
    d.draw(p)
points_right_offset = np.array(points_right_offset)
    
f_left = sp.interpolate.interp1d(*points_left_offset.T, kind="linear")
f_right_inverse = sp.interpolate.interp1d(*np.flip(points_right_offset.T, axis=0), kind="linear")

p = draw.Path(stroke_width = trace_width, stroke="RED", fill_opacity=0)
p.M(*(points_left[0]))
for point in points_left[1:]:
    p.L(*point)
d.draw(p)
outline_left = p
    
p = draw.Path(stroke_width = trace_width, stroke="BLUE", fill_opacity=0)
p.M(*(points_right[0]))
for point in points_right[1:]:
    p.L(*point)
d.draw(p)
outline_right = p


long_width = trace_width*2+trace_spacing*2
x_boxes = np.arange(edge_offset*2, (width/2)-long_width, long_width)


p = draw.Path(stroke_width = trace_width, stroke="green", fill_opacity=0)
p.M(edge_offset*2+trace_width/2,-2*trace_width)
p.L(edge_offset*2+trace_width/2,0)


for i,l in enumerate(x_boxes):
    ideal_height = f_left(l)    
    d.draw(draw.Rectangle(l,0,long_width, ideal_height, fill="orange"))
    
    p.L(l+trace_width/2,ideal_height-trace_width)
    p.A(trace_width/2, trace_width/2, 0, False, False, l+trace_width, ideal_height-trace_width/2)
    p.L(l+trace_width+trace_spacing, ideal_height-trace_width/2)
    p.A(trace_width/2, trace_width/2, 0, False, False, l+trace_width+trace_spacing+trace_width/2, ideal_height-trace_width)
    p.L(l+3*trace_width/2+trace_spacing, 0)
    if not i == len(x_boxes)-1:
        p.A(trace_width/2, trace_width/2, 0, False, True,l+trace_width*2+trace_spacing, -1*trace_width/2)
        p.L(l+trace_width*2+2*trace_spacing, -1*trace_width/2)
        p.A(trace_width/2, trace_width/2, 0, False, True, l+5*trace_width/2+2*trace_spacing, 0)

p.L(l+5*trace_width/2, -2*trace_width)
d.draw(p)
length_left = calc_length(p)
path_left = p

#Use this function instead to make the right side mirror the left side. 
def calc_right_symmetric(bias=0):
    p = draw.Path(stroke_width=trace_width, stroke="green", fill_opacity=0)
    p.M(width - 2 * edge_offset, -2 * trace_width)
    p.L(width - 2 * edge_offset, 0)
 
    # Use x_boxes for the right side, similar to the left side
    x_boxes = np.arange(edge_offset * 2, (width / 2) - long_width, long_width)
    for i, l in enumerate(x_boxes):
        ideal_height = f_left(l)  # Use f_left to get the ideal height for the right side to match the left side
        d.draw(draw.Rectangle(width - l - long_width, 0, long_width, ideal_height, fill="aqua"))
 
        max_step_bias = ((ideal_height + width / 2 - trace_width) - (width / 2 + 4 * trace_width / 2 + trace_spacing)) * 0.75
        use_step_bias = min(bias / 2, max_step_bias)
        bias -= use_step_bias * 2
 
        # Mirror the left side drawing logic for the right side
        p.L(width - l - trace_width / 2, ideal_height - trace_width)
        p.A(trace_width / 2, trace_width / 2, 0, False, True, width - l - trace_width, ideal_height - trace_width / 2)
        p.L(width - l - trace_width - trace_spacing, ideal_height - trace_width / 2)
        p.A(trace_width / 2, trace_width / 2, 0, False, True, width - l - trace_width - trace_spacing - trace_width / 2, ideal_height - trace_width)
        p.L(width - l - 3 * trace_width / 2 - trace_spacing, 0)
 
        if not i == len(x_boxes) - 1:
            p.A(trace_width / 2, trace_width / 2, 0, False, False, width - l - trace_width * 2 - trace_spacing, -1 * trace_width / 2)
            p.L(width - l - trace_width * 2 - 2 * trace_spacing, -1 * trace_width / 2)
            p.A(trace_width / 2, trace_width / 2, 0, False, False, width - l - 5 * trace_width / 2 - 2 * trace_spacing, 0)

    p.L(width - l - 5 * trace_width / 2, -2 * trace_width)
    return p

def calc_right(bias=0):
    p = draw.Path(stroke_width = trace_width, stroke="green", fill_opacity=0)
    p.M(width-2*edge_offset, -2*trace_width)
    p.L(width-2*edge_offset, 0)
    p.A(trace_width/2, trace_width/2, 0, False, True, width-2*edge_offset-trace_width/2, trace_width/2)
    p.L(width/2+2*trace_width+trace_spacing, trace_width/2)
    p.A(trace_width/2, trace_width/2, 0, False, False, width/2+3*trace_width/2+trace_spacing, trace_width)
    p.L( width/2+3*trace_width/2+trace_spacing, trace_width + trace_spacing)
    p.A(trace_width/2, trace_width/2, 0, False, False,  width/2+2*trace_width+trace_spacing, trace_width + trace_width/2 + trace_spacing)

    y_boxes = np.arange(trace_width, height-long_width-edge_offset, long_width)
    for i,l in enumerate(y_boxes):
        ideal_width = f_right_inverse(l+long_width) - width/2
        d.draw(draw.Rectangle(width/2,l ,ideal_width, long_width, fill="aqua"))
        
        max_step_bias = ( (ideal_width + width/2 - trace_width)-(width/2+4*trace_width/2+trace_spacing)  )*0.75
        use_step_bias = min( bias/2, max_step_bias )
        bias -= use_step_bias*2
        
        p.L(ideal_width + width/2 - trace_width - use_step_bias, l + trace_width/2 + trace_spacing)
        p.A(trace_width/2, trace_width/2, 0, False, True,ideal_width + width/2 - trace_width/2 - use_step_bias,  l + trace_width + trace_spacing)
        p.L(ideal_width + width/2 - trace_width/2 - use_step_bias,  l + trace_width + 2*trace_spacing)
        p.A(trace_width/2, trace_width/2, 0, False, True, ideal_width + width/2 - trace_width - use_step_bias,  l + 3*trace_width/2 + 2*trace_spacing)
        p.L(width/2+4*trace_width/2+trace_spacing,  l + 3*trace_width/2 + 2*trace_spacing)
        if not i == len(y_boxes)-1:
            p.A(trace_width/2, trace_width/2, 0, False, False, width/2+3*trace_width/2+trace_spacing,  l + 2*trace_width + 2*trace_spacing)
            p.L(width/2+3*trace_width/2+trace_spacing,  l + 2*trace_width + 3*trace_spacing)
            p.A(trace_width/2, trace_width/2, 0, False, False, width/2+4*trace_width/2+trace_spacing,  l + 5*trace_width/2 + 3*trace_spacing)

    p.L(width/2+trace_width,  l + 3*trace_width/2 + 2*trace_spacing)
    p.A(trace_width/2, trace_width/2, 0, False, True, width/2+trace_width/2,  l + trace_width + 2*trace_spacing)
    p.L(width/2+trace_width/2,-2*trace_width)
    return p

p = (calc_right_symmetric if make_symmetric else calc_right)()
#d.draw(p)    
length_right = calc_length(p)
print("len left:{:.2f}, len right: {:.2f}".format(length_left, length_right))
assert length_right >= length_left, "Right side too short!"
p = (calc_right_symmetric if make_symmetric else calc_right)(bias=length_right-length_left)
d.draw(p)
path_right = p
print("final length tune diff: {:.2f}".format(calc_length(p)-length_left))

p = draw.Path(stroke_width = trace_width, stroke="pink", fill_opacity=0)
p.M(width-2*edge_offset, -2*trace_width)
p.L(width-2*edge_offset, 0)
p.A(trace_width/2, trace_width/2, 0, False, True, width-2*edge_offset-trace_width/2, trace_width/2)
p.L(width/2+2*trace_width+trace_spacing, trace_width/2)
p.A(trace_width/2, trace_width/2, 0, False, False, width/2+3*trace_width/2+trace_spacing, trace_width)
p.L( width/2+3*trace_width/2+trace_spacing, 15)
#d.draw(p)    


d.draw(draw.Circle(*nodes_left[:,1], trace_width*2, fill="pink"))
d.draw(draw.Circle(*nodes_left[:,2], trace_width*2, fill="pink"))
d.draw(draw.Circle(*nodes_right[:,1], trace_width*2, fill="pink"))
d.draw(draw.Circle(*nodes_right[:,2], trace_width*2, fill="pink"))

d.save_svg('curve_verbose.svg')
d.rasterize() 
d.set_pixel_scale(image_scale)
d


q = draw.Drawing(800, 1000, origin=(-10,-10), displayInline=False)
q.draw(outline_left)
q.draw(outline_right)
q.draw(path_left)
q.draw(path_right)
q.save_svg("curve_simplified.svg")


len left:1811.17, len right: 1948.85
final length tune diff: -0.00
